In [ ]:
import pandas as pd 
import numpy as np 
import datetime as dt

In [ ]:
df_train = pd.read_csv('data/train.csv')
df_store = pd.read_csv('data/store.csv')
df_holdout = pd.read_csv('data/holdout.csv')

In [ ]:
def merge_data(train, store):
        
        """ Takes two dataframes,
            creates two copies
            drop the customers axis
            drop the nan for sale and stores
            make sure the store coumns are of the same type. 
            inner merge on the column store.  """
        train_copy = train.copy()
        store_copy = store.copy()
        train_copy = train_copy.drop(columns = ['Customers'])
        train_copy = train_copy.dropna(axis = 0, how = 'any', subset = ['Sales', 'Store'])
        train_copy['Store'] = train_copy['Store'].astype(int)
        store_copy['Store'] = store_copy['Store'].astype(int)
        df_p = pd.merge(train_copy, store_copy, how = 'inner', on = 'Store')
    
        return df_p

In [ ]:
hot_encoded_columns = ['Open', 'StateHoliday', 'StoreType', 'Assortment']

dropped_columns = ['Store', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear',\
                   'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']

filled_in_median = ['CompetitionDistance']

filled_in_mode = ['Promo', 'SchoolHoliday']
 
target = ['Sales']

In [ ]:
df_p = merge_data(df_train, df_store)


In [ ]:
dropped_columns_n = ['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear',\
                   'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']
df_p1 = df_p.drop(columns = dropped_columns_n)
df_p1.head(3)

In [ ]:
# Estimate day-month-year etc

df_copy = df_p1.copy()
df_copy['Date'] = pd.to_datetime(df_copy['Date'])    
df_copy['day'] = df_copy['Date'].dt.day
df_copy['month'] = df_copy['Date'].dt.month
df_copy['year'] = df_copy['Date'].dt.year
df_copy['weekday_name'] = df_copy['Date'].dt.day_name()
df_copy['weekday'] = df_copy['Date'].apply(lambda x: x.weekday())


In [ ]:
# Fill empty with "9" -> set these as string to make plotting easier
df_copy["Promo"] = df_copy["Promo"].fillna(9).astype(str)
df_copy["Open"] = df_copy["Open"].fillna(9).astype(str)
df_copy["SchoolHoliday"] = df_copy["SchoolHoliday"].fillna(9).astype(str)
df_copy.head(3)



In [ ]:
#fig.add_scatter(x=x, y=np.sin(x), name='sin(x)', row=1, col=1)


In [ ]:
#from plotly.subplots import make_subplots
#import plotly.graph_objects as go

#fig = make_subplots(rows=1, cols=2) 

#fig.add_box(x=df_copy["StoreType"], 
#            y=df_copy["Sales"],
#            row=1, col=1)\
#    .add_box(x=df_copy["Assortment"], 
#            y=df_copy["Sales"],
#            row=1, col=2)

In [ ]:
# Plot Sales per StoreType
import plotly.express as px

px.box(
    df_copy, 
    x="StoreType", 
    y="Sales", 
    color="StoreType",
    width=500,
    height=500,
    title='Sales per StoreType',
    points=False,
)

In [ ]:
# Sales per Assortment 
px.box(
    df_copy, 
    x="Assortment", 
    y="Sales", 
    color="Assortment",
    width=500,
    height=500,
    title='Sales per Assortment',
    points=False,
)

In [ ]:

# # Any relation of Store Type & Assortment ?
px.box(
    df_copy, 
    x="Assortment", 
    y="Sales", 
    color="Assortment",
    facet_col="StoreType",
    width=950,
    height=500,
    title='Sales per Assortment',
    points=False,
)

# NOTE: 1) Assortment *c* sales a lot in StoreType=b
# 2) Assortment *b* is only found in StoreType=b

In [ ]:
# Any relation of Store Type & Assortment ?
unique_storetypes = df_copy["StoreType"].unique()

dict_storetype_assortment = {}
for storetype in unique_storetypes:
    dict_storetype_assortment[storetype] = df_copy\
                                            .query(f"StoreType=='{storetype}'")\
                                            .loc[:, "Assortment"]\
                                            .value_counts()\
                                            .to_dict()


dict_storetype_assortment
for k,v in dict_storetype_assortment.items():
    print(f"\nStore type *{k}* ->  assortments: {v}")

# Note: Apparently StoreType *b* is the only one selling assortment *b*

In [ ]:
# Sales for School Holidays
px.box(
    df_copy, 
    x="SchoolHoliday", 
    y="Sales",
    color="SchoolHoliday",
    width=500,
    height=500,
    title='Sales for SchoolHoliday',
    points=False,
)

# NOTE: Missing values look more similar to no-holidays 
# -> Impute them as 0?

In [ ]:
# Sales for Regional-Promo
px.box(
    df_copy, 
    x="Promo", 
    y="Sales",
    color="Promo",
    width=500,
    height=500,
    title='Sales for Regional Promo',
    points=False,
)

# NOTE: Regional-promo works. Higher sales when it's running.

In [ ]:
# Sales for Overal-Promo
#df_copy.groupby("Promo2")["Sales"].mean()

px.box(
    df_copy, 
    x="Promo2", 
    y="Sales",
    color="Promo2",
    width=500,
    height=500,
    title='Sales for Overal Promo',
    points=False,
)

# NOTE: Overall-promo seems (ineffective to) counter-productive. More sales when a shop is NOT participating.

In [ ]:
# Sales per Week Day 

df_copy.groupby(["weekday"])["Sales"].describe()

In [ ]:
df_copy.groupby(["weekday_name"])["Sales"].describe()

In [ ]:
# Sales per Week Day 

px.box(
    df_copy, 
    x="weekday_name", 
    y="Sales", 
    color="weekday_name",
    width=500,
    height=500,
    title='Sales per WeekDay',
    points=False
)

#NOTE: Clear, ~ no sales on Sunday, high sales on Monday

In [ ]:
# Sales per WeekDay per Store Type
px.box(
    df_copy, 
    x="weekday_name", 
    y="Sales", 
    color="StoreType",
    width=900,
    height=500,
    title='Sales per Store Type',
)

# NOTE: So StoreType *b* is open on Sundays

In [ ]:
# Sales per Weekday per Assortment 
px.box(
    df_copy, 
    x="weekday_name", 
    y="Sales", 
    color="Assortment",
    width=900,
    height=500,
    title='Sales per Weekday per Assortment',
)

In [ ]:
# Sales per WeekDay per Overal Promo
px.box(
    df_copy, 
    x="weekday_name", 
    y="Sales",
    color="Promo2",
    width=800,
    height=500,
    title='Sales per WeekDay per Overall Promo',
    points=False,
)

In [ ]:
df_copy["Promo"].isnull().sum()

In [ ]:
# Sales per WeekDay per Regional-Promo
px.box(
    df_copy, 
    x="weekday_name", 
    y="Sales",
    color="Promo",
    width=800,
    height=500,
    title='Sales per WeekDay per Regional Promo',
    points=False,
)

In [ ]:
# Sales per month day

px.box(
    df_copy, 
    x="day", 
    y="Sales", 
    color="day",
    width=1000,
    height=500,
    title='Sales per month day',
    points=False
)

# NOTE : seems to have some periodicity with peaks at start,middle & end of month

In [ ]:
# Sales per Month
df_copy.groupby(["month"])["Sales"].describe()

In [ ]:
# Sales per Month
px.box(
    df_copy, 
    x="month", 
    y="Sales", 
    color="month",
    width=700,
    height=550,
    title='Sales per Month',
    points=False
)

# NOTE: as expected higher sales for December

In [ ]:
# Sales per Month per Store Type
px.box(
    df_copy, 
    x="month", 
    y="Sales", 
    color="StoreType",
    width=900,
    height=500,
    title='Sales per Month per Store Type',
)

In [ ]:
# Sales per Month per Assortment
px.box(
    df_copy, 
    x="month", 
    y="Sales", 
    color="Assortment",
    width=900,
    height=500,
    title='Sales per Month per Assortment',
)

In [ ]:
df_copy.columns


In [ ]:
df_grpday_sortA = pd.DataFrame(\
                    df_copy\
                    .query("StoreType=='a'")\
                    .groupby(["Date"])["Sales"]\
                    .mean())\
                    .assign(StoreType="a")
df_grpday_sortA

In [ ]:
# Any relation of Store Type & Assortment ?

unique_storetypes = df_copy["StoreType"].unique()

dict_storetype_assortment = {}
for storetype in unique_storetypes:
    dict_storetype_assortment[storetype] = df_copy\
                                            .query(f"StoreType=='{storetype}'")\
                                            .loc[:, "Assortment"]\
                                            .value_counts()\
                                            .to_dict()


dict_storetype_assortment
for k,v in dict_storetype_assortment.items():
    print(f"\nStore type *{k}* ->  assortments: {v}")

# Note: Apparently StoreType *b* is the only one selling assortment *b*

In [ ]:
# Time-series of Sales for every Store Type

def groupdate_sales_storetype(df, storetype:str):
    """ Get dataframe with avgSales and StoreType grouped by Date """
    df_n = pd.DataFrame(df\
                    .query(f"StoreType=='{storetype}'")\
                    .groupby(["Date"])["Sales"]\
                    .mean())\
                    .assign(StoreType=storetype)
    return df_n   

In [ ]:
# a. For each StoreType : Get dataframe with avgSales and StoreType grouped by Date"""
# b. Concatenate across rows 

unique_storetypes = df_copy["StoreType"].unique()

df_grpday_sort = pd.concat(
    [groupdate_sales_storetype(df_copy, storetype) \
     for storetype in unique_storetypes], axis=0
)

df_grpday_sort

In [ ]:
# Grouping by date
#df_grpday_sortA = df_copy.query("SortType=='a'").groupby(["Date"]).mean()

#df_grpday = df_copy.groupby(["Date"]).mean()
#df_grpday

#df_grpday.index

In [ ]:
# Time-series of Sales for every Store Type
px.scatter(
    df_grpday_sort,
    x=df_grpday_sort.index, 
    y="Sales",
    color="StoreType",
    width=950,
    height=500,
    title='Sales over time per StoreType')\
    .update_traces(mode='lines+markers')


In [ ]:
df_copy

In [ ]:
# Sales ~ CompetitionDistance


In [ ]:
unique_storetypes = df_copy["StoreType"].unique()
unique_storetypes

In [ ]:
# Time-series of Sales for every Store Type

def groupstore_sales_compdist(df, storetype:str):
    """ Get dataframe with avgSales and CompetitionDistance grouped by Store"""
    df_n = pd.DataFrame(df\
                    .query(f"StoreType=='{storetype}'")\
                    .groupby(["Store"])["Sales", "CompetitionDistance"]\
                    .mean())\
                    .assign(StoreType=storetype)
    return df_n   

In [ ]:
unique_storetypes = df_p["StoreType"].unique()

df_grpstore_sales_compdist = pd.concat(
    [groupstore_sales_compdist(df_p, storetype) \
     for storetype in unique_storetypes], axis=0
)

df_grpstore_sales_compdist

In [ ]:
# Some checking

A = df_copy["CompetitionDistance"].sort_values().unique()
B = df_grpstore_sales_compdist["CompetitionDistance"].sort_values().unique()

C = np.array(A==B)
C
#print(A[-1], B[-1])

In [ ]:
# Spearman Correlation between Sales ~ CompetitionDistance

In [ ]:
corr_spearman = df_grpstore_sales_compdist.corr(method="spearman")
corr_spearman

In [ ]:
unique_storetypes = df_p["StoreType"].unique()

for storetype in unique_storetypes:
    corr_spearman = df_grpstore_sales_compdist\
                        .query(f"StoreType=='{storetype}'")\
                        .corr(method="spearman")
    print(f"STORETYPE *{storetype}*:\n{corr_spearman}\n")


In [ ]:
#df_grpstore_sales_compdist_n["StoreType"].value_counts()

In [ ]:

px.scatter(
    df_grpstore_sales_compdist,
    x="CompetitionDistance", 
    y="Sales",
    color="StoreType",
    trendline="ols",
    width=600,
    height=600,
    title='Sales ~ CompetitionDistance')\

